# Scikit-Learn机器学习快速入门与实践

### API一致性示例
- Estimator支持fit()、transform()、predict()、fit_transform()等统一的方法。
- Estimator包括模型、数据转换、Pipeline等。

In [ ]:
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score

# 1. 分类任务
# 生成分类数据集
X_clf, y_clf = make_classification(n_samples=100, n_features=4, n_classes=2, random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# 分类模型：LogisticRegression
clf = LogisticRegression()
clf.fit(X_clf_train, y_clf_train)  # 统一 fit 接口
y_clf_pred = clf.predict(X_clf_test)  # 统一 predict 接口
clf_score = accuracy_score(y_clf_test, y_clf_pred)  # 评估
print("Classification Accuracy:", clf_score)

# 2. 回归任务
# 生成回归数据集
X_reg, y_reg = make_regression(n_samples=100, n_features=4, random_state=42)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# 回归模型：RandomForestRegressor
reg = RandomForestRegressor(random_state=42)
reg.fit(X_reg_train, y_reg_train)  # 统一 fit 接口
y_reg_pred = reg.predict(X_reg_test)  # 统一 predict 接口
reg_score = r2_score(y_reg_test, y_reg_pred)  # 评估
print("Regression R² Score:", reg_score)

# 3. 数据转换任务
# 使用分类数据集进行数据标准化
scaler = StandardScaler()
scaler.fit(X_clf_train)  # 统一 fit 接口
X_clf_train_scaled = scaler.transform(X_clf_train)  # 统一 transform 接口
X_clf_test_scaled = scaler.transform(X_clf_test)
print("Scaled Data (first sample):", X_clf_test_scaled[0])


## Scikit-Learn基础实践

### 数据转换示例
- 数据标准化StandardScaler()转换后的效果。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 创建一个简单的带标签分类数据集
# 特征：2维（身高、体重），标签：0（负类）或1（正类）
X = np.array([[170, 60], [165, 55], [180, 80], [175, 70], [160, 50], [185, 85]])
y = np.array([0, 0, 1, 1, 0, 1])

# 初始化 StandardScaler
scaler = StandardScaler()

# 对特征数据进行标准化转换
X_transformed = scaler.fit_transform(X)

# 打印原始数据和转换后的数据
print("原始特征数据：")
print(X)
print("\n原始标签：")
print(y)
print("\n转换后的特征数据：")
print(X_transformed)

### Imputer数据转换
- SimpleImputer数据预处理组件，位于 klearn.impute模块，用于处理数据集中的缺失值（NaN或None）
- 主要功能是通过指定的策略自动填充缺失值，以确保数据集完整，适合后续的机器学习模型训练
- 支持填充策略：
  - 均值填充（strategy='mean'）：用特征列的均值替换该列中的缺失值（仅适用于数值型数据）
  - 中位数填充（strategy='median'）：用特征列的中位数替换缺失值（适用于数值型数据，特别对异常值敏感的数据集）
  - 众数填充（strategy='most_frequent'）：用特征列出现频率最高的值替换缺失值（适用于数值型或类别型数据）
  - 常数填充（strategy='constant'）：用用户指定的固定值（通过 fill_value 参数设置）替换缺失值（适用于数值型或类别型数据）

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 创建一个简单的带标签分类数据集
# 特征：2维（身高、体重），标签：0（负类）或1（正类）
X = np.array([[170, 60], [165, 55], [180, None], [175, 70], [160, 50], [185, 85]])
y = np.array([0, 0, 1, 1, 0, 1])

# 使用 SimpleImputer 进行均值填充
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X_imputed)

# 打印原始数据和转换后的数据
print("原始特征数据：")
print(X)
print("\n原始标签：")
print(y)
print("\nimputer转换后的特征数据：")
print(X_imputed)
print("\nstandardscaler转换后的特征数据：")
print(X_transformed)

### 基于上面的数据样本训练一个分类模型（逻辑回归）

本示例选用LogisticRegression算法，尽管名字中带有“回归”，但它实际上是一种分类算法，主要用于解决二分类问题（也可以扩展到多分类）。它基于线性回归的原理，通过Sigmoid函数 将线性回归的输出映射到 (0, 1) 的概率值，从而进行分类。

**工作原理**

1. 线性组合： 输入特征$X$首先进行线性组合，得到一个线性分数$z$：   

   $z=w_0+w_1x_1+w_2x_2+...+w_nw_n$

   其中，$w$是权重，$x$是特征，$w_0$是截距。
2. Sigmoid函数 (S函数)： 将z输入到Sigmoid函数中，将其转换为一个介于0到1之间的概率值$P(y=1∣X)$：

   $P(y=1|x)=\frac{1}{1+e^{-z}}$
3. 分类决策： 通常，如果$P(y=1∣X)0.5$，则预测为正类；否则，预测为负类。
4. 损失函数： 逻辑回归通常使用对数损失 (Log Loss) 或交叉熵损失 (Cross-Entropy Loss) 来衡量模型预测与真实标签之间的差距，并通过梯度下降等优化算法来最小化损失函数，从而找到最佳的权重 w。

**常用参数**
- penalty: 正则化类型 ('l1', 'l2', 'elasticnet', 'none')。
- C: 正则化强度的倒数，C 越小表示正则化越强。
- solver: 优化算法 ('liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga')。
- max_iter: 最大迭代次数。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression # 导入LogisticRegression

# 创建一个简单的带标签分类数据集
# 特征：2维（身高、体重），标签：0（负类）或1（正类）
X = np.array([[170, 60], [165, 55], [180, None], [175, 70], [160, 50], [185, 85]])
y = np.array([0, 0, 1, 1, 0, 1])

# 使用SimpleImputer 进行均值填充
imputer =SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X_imputed)

# 初始化 LogisticRegression 模型
model = LogisticRegression(random_state=42)

# 训练模型
model.fit(X_transformed, y)

# 创建两个专门用于测试的样本，并进行数据转换
y_test = np.array([[172, 55], [178, 75]])
y_test_scaled = scaler.transform(X_test)

# 对测试样本进行预测
y_pred = model.predict(y_test_scaled)

# 打印结果
print("\n测试样本特征数据：")
print(X_test)
print("\n标准化后的测试样本特征数据：")
print(X_test_scaled)
print("\n测试样本预测标签：")
print(y_test_pred)

### 另一个分类模型SVC

支持向量机 (Support Vector Machine, SVM) 是一种强大而灵活的分类（和回归）算法。对于分类任务，它被称为Support Vector Classifier(SVC)。SVC的核心思想是找到一个最优的超平面 (Hyperplane)，将不同类别的数据点分隔开，并且使这个超平面到最近数据点（支持向量）的距离最大化，这个距离被称为 间隔 (Margin)。

**常用参数**
- C: 惩罚参数，C越大，对错误分类的惩罚越大。
- kernel: 核函数类型 ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed')。
- degree: 多项式核的次数。
- gamma: RBF、Poly和Sigmoid核函数的核系数。'scale'（默认）表示 1/(n_features * X.var())，'auto' 表示 1/n_features。
- probability: 是否启用概率估计（会增加计算开销）。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC   # 导入SVC

# 创建一个简单的带标签分类数据集
# 特征：2维（身高、体重），标签：0（负类）或1（正类）
X = np.array([[170, 60], [165, 55], [180, None], [175, 70], [160, 50], [185, 85]])
y = np.array([0, 0, 1, 1, 0, 1])

# 使用 SimpleImputer 进行均值填充
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X_imputed)

# 初始化 SVC 模型
model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42, probability=True)

# 训练模型
model.fit(X_transformed, y)

# 创建两个专门用于测试的样本，并进行数据转换
X_test = np.array([[172, 55], [178, 75]])
X_test_scaled = scaler.transform(X_test)

# 对测试样本进行预测
y_test_pred = model.predict(X_test_scaled)

# 打印结果
print("\n测试样本预测标签：")
print(y_test_pred)

### 基于类似上面的分类数据集进行机器学习
将扩展之前的分类数据集到200个样本，使用scikit-learn的LogisticRegression模型进行训练和评估，数据集将分为90%的训练集和10%的测试集。
- 生成一个扩展的分类数据集（200个样本），特征为身高（160-190cm）和体重（50-90kg），标签基于规则（身高+体重>250为1，否则为0）并添加随机噪声。
- 打印训练集和测试集的前5行特征数据、标签、标准化后的特征数据、预测标签，以及训练集和测试集的准确率和测试集的分类报告。

#### 评估指标
在机器学习中，评估指标 (Evaluation Metrics) 是用来衡量模型性能好坏的标准。它们提供了一种量化的方式，帮助我们理解模型在处理新数据时的表现。简单来说，评估指标就是一套衡量模型“好坏”的打分规则。

**为什么要获取评估指标？**

模型在训练完成后，我们通常会使用未参与训练的测试集 (Test Set) 对其进行评估，并获取各项评估指标。这样做至关重要，原因如下：
- 量化模型性能： 评估指标将抽象的模型表现转化为具体的数值，例如准确率是 85%，或者 F1 分数是 0.78。这使得我们可以客观地比较不同模型、不同参数设置下的性能。
- 指导模型选择与优化： 没有评估指标，我们就无法知道当前模型是否足够好，或者哪个模型更适合我们的任务。通过评估指标，我们可以判断模型是否存在过拟合或欠拟合，从而指导我们进行特征工程、模型选择或超参数调优。
- 避免过拟合： 模型在训练集上表现完美，但在测试集上却很糟糕，这通常意味着模型过拟合了。评估指标在测试集上的表现能有效揭示过拟合现象，确保模型的泛化能力，即对未知数据的处理能力。
- 业务决策依据： 不同的业务场景对模型性能有不同的侧重。例如，在疾病诊断中，我们可能更关心召回率（不错过任何病人），而在垃圾邮件识别中，可能更关心精确率（不要把正常邮件误判为垃圾邮件）。评估指标帮助我们将模型性能与实际业务目标对齐。
- 沟通与协作： 评估指标提供了一种标准化的语言，让数据科学家、工程师和业务人员能够清晰地沟通模型的能力和局限性。

**分类模型的常用指标**
1. 混淆矩阵 (Confusion Matrix)
   混淆矩阵是一个表格，可视化了模型预测的类别与真实类别的对应关系。它包含了以下四个基本指标：
   - 真阳性 (TP-True Positive)： 真实为正类，预测也为正类。
   - 真阴性 (TN-True Negative)： 真实为负类，预测也为负类。
   - 假阳性 (FP-False Positive)： 真实为负类，预测为正类（I类错误）。
   - 假阴性 (FN-False Negative)： 真实为正类，预测为负类（II类错误）。
2. 准确率 (Accuracy)
   准确率是最直观的指标，表示模型正确预测的样本数量占总样本数量的比例。

   $Accuracy=\frac{TP+TN}{TP+FP+TN+FN}$
   - 优点：易于理解和计算
   - 缺点：在类别不平衡 (Class Imbalance) 的数据集中容易产生误导。例如，在一个95%是负类的场景中，一个总是预测为负类的模型也能达到95%的准确率，但这显然不是一个好模型。
3. 精确率 (Precision)
   精确率衡量的是模型预测为正类的样本中，真正是正类的比例。
   
   $Precesion=\frac{TP}{TP+FP}$
   - 适用场景： 当假阳性 (False Positive) 的成本很高时。
   - 例如，垃圾邮件分类（我们不希望将正常邮件标记为垃圾邮件），或者癌症诊断（我们不希望误诊健康人为患病）。
4. 召回率 (Recall) / 灵敏度 (Sensitivity)
   召回率衡量的是所有真正是正类的样本中，被模型正确预测为正类的比例。
   $Recall=\frac{TP}{TP+FN}$
   - 适用场景： 当假阴性 (False Negative) 的成本很高时。
   - 例如，疾病诊断（我们不希望漏诊病人），或者欺诈检测（我们不希望错过真正的欺诈行为）。
5. F1-Score
   F1-Score 是精确率和召回率的调和平均值。它提供了一个综合考虑两者的单一指标。
   
   $F1-Score=2 \times \frac{Precision×Recall}{Precision+Recall}$
   - 优点： 在类别不平衡的数据集中比准确率更有用，因为它同时考虑了假阳性和假阴性。
   - 适用场景： 当我们希望精确率和召回率都较高，并且它们之间存在一个平衡时。

**分类评估函数**
- accuracy_score：计算分类准确率，参数为测试集的真实标签和预测结果（y_true和y_pred），例如accuracy_score(y_true, y_pred)。
- precision_score, recall_score, f1_score：分别计算精确率、召回率和 F1-Score。参数为测试集的真实标签和预测结果（y_true和y_pred）。
- classification_report：生成一个包含精确率、召回率、F1-Score 和支持度（Support）的文本报告。参数为测试集的真实标签和预测结果（y_true和y_pred）。
- confusion_matrix：计算混淆矩阵。参数为测试集的真实标签和预测结果（y_true和y_pred）。
- roc_curve 和 roc_auc_score：计算 ROC 曲线点和 ROC AUC 值。参数为测试集的真实标签和预测的结果概率(y_true, y_proba)。
  - roc_curve： 接收真实标签和预测概率（或决策函数值），返回真阳性率 (TPR) 和假阳性率 (FPR) 的序列，用于绘制 ROC 曲线。
  - roc_auc_score： 计算 ROC 曲线下的面积 (AUC)。AUC 衡量了模型区分正负类的能力，值越接近 1 越好，0.5 表示随机分类。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子以确保结果可重现
np.random.seed(42)

# 生成扩展的分类数据集（200个样本）
# 特征：身高（160-190cm），体重（50-90kg），标签：0或1
n_samples = 200
height = np.random.uniform(160, 190, n_samples)
weight = np.random.uniform(50, 90, n_samples)
X = np.column_stack((height, weight))
# 生成标签：基于简单的规则（例如，身高+体重>250则为1，否则为0）并添加噪声
y = np.where(height + weight > 250, 1, 0)
# 添加一些随机噪声
noise = np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1])
y = np.logical_xor(y, noise).astype(int)

# 划分训练集和测试集（90%训练，10%测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 初始化 StandardScaler 并对特征数据进行标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 初始化 LogisticRegression 模型
model = LogisticRegression(random_state=42)

# 训练模型
model.fit(X_train_scaled, y_train)

# 预测训练集和测试集
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# 计算准确率
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 打印结果
print("训练集特征数据（前5行）：")
print(X_train[:5])
print("\n测试集特征数据（前5行）：")
print(X_test[:5])
print("\n训练集标签（前5个）：")
print(y_train[:5])
print("\n测试集标签（前5个）：")
print(y_test[:5])
print("\n训练集标准化后的特征数据（前5行）：")
print(X_train_scaled[:5])
print("\n测试集标准化后的特征数据（前5行）：")
print(X_test_scaled[:5])
print("\n训练集预测标签（前5个）：")
print(y_train_pred[:5])
print("\n测试集预测标签（前5个）：")
print(y_test_pred[:5])
print("\n训练集准确率：")
print(f"{train_accuracy:.2f}")
print("\n测试集准确率：")
print(f"{test_accuracy:.2f}")
print("\n测试集分类报告：")
print(classification_report(y_test, y_test_pred))

### 交叉验证示例

**工作原理**
1. 将原始数据集划分为K个（通常是5或10个）大小相似的互斥子集（或称为“折叠”，folds）。
2. 在每次迭代中，选择其中一个子集作为验证集 (Validation Set)，而将其余K−1个子集合并起来作为训练集 (Training Set)。
3. 使用训练集训练模型。
4. 使用验证集评估训练好的模型，记录评估指标（如准确率、F1分数等）。
5. 重复步骤2-4共K次，确保每个子集都作为验证集使用一次。
6. 最终，将K次评估结果取平均值（或中位数、标准差等），作为模型性能的最终估计。

**常用的交叉验证方法**
1. K-Fold Cross-Validation (K 折交叉验证)
3. Stratified K-Fold Cross-Validation (分层 K 折交叉验证)
4. Leave-One-Out Cross-Validation (LOOCV)
5. Shuffle-Split Cross-Validation (洗牌-分割交叉验证)
6. GroupKFold (分组 K 折交叉验证)

**交叉验证函数**
1. cross_val_score
2. cross_validation

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子以确保结果可重现
np.random.seed(42)

# 生成扩展的分类数据集（200个样本）
# 特征：身高（160-190cm），体重（50-90kg），标签：0或1
n_samples = 200
height = np.random.uniform(160, 190, n_samples)
weight = np.random.uniform(50, 90, n_samples)
X = np.column_stack((height, weight))

# 随机将约3%的数据设置为NaN
mask = np.random.random(X.shape) < 0.03
X[mask] = np.nan

# 生成标签：基于简单的规则（例如，身高+体重>250则为1，否则为0）并添加噪声
y = np.where(np.nansum(X, axis=1) > 250, 1, 0)
# 添加一些随机噪声
noise = np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1])
y = np.logical_xor(y, noise).astype(int)

# 划分训练集和测试集（90%训练，10%测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 使用 SimpleImputer 进行均值填充
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# 初始化 LogisticRegression 模型
model = LogisticRegression(random_state=42)

# 进行 5 折交叉验证
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')

# 训练模型
model.fit(X_train_scaled, y_train)

# 预测训练集和测试集
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# 计算准确率
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 打印结果
print("\n交叉验证准确率（每折）：")
print(cv_scores)
print("\n平均交叉验证准确率：")
print(f"{np.mean(cv_scores):.2f} ± {np.std(cv_scores):.2f}")
print("\n训练集准确率：")
print(f"{train_accuracy:.2f}")
print("\n测试集准确率：")
print(f"{test_accuracy:.2f}")
print("\n测试集分类报告：")
print(classification_report(y_test, y_test_pred))

### 模型选择
模型选择不仅仅是挑选算法（例如，是使用逻辑回归还是随机森林），更重要的是找到最适合当前数据集和业务目标的模型及其最佳配置（超参数）。它是一个迭代的过程，通常涉及以下几个方面：
1. 选择模型类型： 决定使用哪种机器学习算法（例如，分类问题是使用支持向量机、决策树还是集成学习）。
2. 选择模型超参数： 每种模型都有其特定的超参数（Hyperparameters），这些参数在模型训练之前设置，而不是通过训练数据学习。例如，随机森林中树的数量 (n_estimators)、支持向量机中的正则化参数 C 和核函数 (kernel) 等。选择合适的超参数对模型性能至关重要。
3. 评估模型性能： 使用适当的评估指标（如准确率、F1 分数、AUC 等）来量化模型的表现。
4. 避免过拟合和欠拟合： 模型选择的目标是找到一个既能很好地拟合训练数据，又能很好地泛化到新数据的模型。过拟合（模型在训练数据上表现很好，但在新数据上很差）和欠拟合（模型在训练数据和新数据上都表现不佳）是模型选择过程中需要避免的常见问题。

**模型选择机制**
1. 网格搜索 (Grid Search)：在预定义的超参数空间中，尝试所有可能的参数组合，并通过交叉验证评估每种组合的性能，最终选择性能最佳的组合。
   - 优点： 简单直观，能确保找到给定参数空间内的最佳组合。
   - 缺点： 计算成本高昂，特别是当参数数量多、每个参数的取值范围大时，搜索空间会呈指数级增长。
2. 随机搜索 (Randomized Search)：在计算资源有限的情况下，通常能比网格搜索更快地找到较好的参数组合，因为它不局限于穷举所有组合。
   - 优点： 效率更高，更适合参数空间很大的情况。
   - 缺点： 不保证找到全局最优解，但通常能找到一个非常好的近似解。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子以确保结果可重现
np.random.seed(42)

# 生成扩展的分类数据集（200个样本）
# 特征：身高（160-190cm），体重（50-90kg），标签：0或1
n_samples = 200
height = np.random.uniform(160, 190, n_samples)
weight = np.random.uniform(50, 90, n_samples)
X = np.column_stack((height, weight))

# 随机将约3%的数据设置为NaN
mask = np.random.random(X.shape) < 0.03
X[mask] = np.nan

# 生成标签：基于简单的规则（例如，身高+体重>250则为1，否则为0）并添加噪声
y = np.where(np.nansum(X, axis=1) > 250, 1, 0)
# 添加一些随机噪声
noise = np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1])
y = np.logical_xor(y, noise).astype(int)

# 划分训练集和测试集（90%训练，10%测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 使用 SimpleImputer 进行均值填充
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# 初始化 LogisticRegression 模型
model = LogisticRegression(random_state=42)

# 定义 GridSearchCV 的超参数网格
param_grid = {
    'C': [0.1, 1.0, 10.0],  # 正则化参数
    'solver': ['lbfgs', 'liblinear']  # 优化器
}

# 初始化 GridSearchCV
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=5,  # 5 折交叉验证
    scoring='accuracy',
    n_jobs=-1  # 使用所有可用 CPU 核心
)

# 训练 GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# 获取最佳模型
best_model = grid_search.best_estimator_

# 预测训练集和测试集
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)

# 计算准确率
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 打印结果
print("\n测试集分类报告：")
print(classification_report(y_test, y_test_pred))
print("\n最佳超参数：")
print(grid_search.best_params_)
print("\n最佳交叉验证准确率：")
print(f"{grid_search.best_score_:.2f}")

### 使用Pipeline构建模型训练流水线

在机器学习中，我们通常需要对数据进行一系列的预处理步骤（如特征缩放、缺失值填充、特征选择等），然后再将处理后的数据送入模型进行训练。这个过程如果手动一步步操作，不仅繁琐，而且容易出错，尤其是在交叉验证时，需要确保每个步骤都正确地应用于训练集和测试集。Scikit-learn的Pipeline就是为了解决这个问题而设计的，它能够将多个数据预处理和模型训练步骤串联起来，形成一个统一的工作流。
- Pipeline是Scikit-learn中的一个工具，它允许我们将多个转换器（transformers）和最后一个估计器（estimator）按顺序组合起来。
  - 转换器 (Transformers): 负责数据的转换，例如StandardScaler（标准化）、MinMaxScaler（归一化）、SimpleImputer（缺失值填充）、OneHotEncoder（独热编码）等。它们都实现了fit和transform方法。
  - 估计器 (Estimators): 通常是机器学习模型，例如LogisticRegression（逻辑回归）、RandomForestClassifier（随机森林分类器）、SVC（支持向量机）等。它们实现了fit和predict方法（或predict_proba、decision_function等）。
- 通过Pipeline，我们可以将整个机器学习流程封装成一个单一的estimator对象
  - 对Pipeline对象调用fit方法时，数据会依次经过每个转换器进行处理，然后最终被送入估计器进行训练
  - 调用predict或transform方法时，数据也会按照相同的顺序进行处理

创建一个Pipeline非常简单，只需要传入一个元组列表，每个元组包含两个元素：一个自定义的名称和对应的转换器/估计器对象。

**示例**：
- 生成类似前面的数据集，但在其中随机生成3%左右的空值，这些空值需要一个专门的SimpleImputer进行处理。
- 因此，Pipeline依次包含有如下几个步骤：
  - SimpleImputer（使用均值填充 NaN 值，strategy='mean'）。
  - StandardScaler（标准化特征数据）。
  - LogisticRegression（分类模型，random_state=42）。
- 通过Pipeline的fit方法自动处理缺失值填充、标准化和模型训练。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

# 设置随机种子以确保结果可重现
np.random.seed(42)

# 生成扩展的分类数据集（200个样本）
# 特征：身高（160-190cm），体重（50-90kg），标签：0或1
n_samples = 200
height = np.random.uniform(160, 190, n_samples)
weight = np.random.uniform(50, 90, n_samples)
X = np.column_stack((height, weight))

# 随机将约3%的数据设置为NaN
mask = np.random.random(X.shape) < 0.03
X[mask] = np.nan

# 生成标签：基于简单的规则（例如，身高+体重>250则为1，否则为0）并添加噪声
# 忽略NaN进行计算
y = np.where(np.nansum(X, axis=1) > 250, 1, 0)
# 添加一些随机噪声
noise = np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1])
y = np.logical_xor(y, noise).astype(int)

# 划分训练集和测试集（90%训练，10%测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 创建 Pipeline，整合 SimpleImputer、StandardScaler 和 LogisticRegression
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42))
])

# 训练 Pipeline
pipeline.fit(X_train, y_train)

# 预测训练集和测试集
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# 计算准确率
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 打印结果
print("\n训练集准确率：")
print(f"{train_accuracy:.2f}")
print("\n测试集准确率：")
print(f"{test_accuracy:.2f}")
print("\n测试集分类报告：")
print(classification_report(y_test, y_test_pred))

### 综合案例
综合交叉验证、模型选择、Pipeline等功能于一体的综合案例。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.pipeline import Pipeline

# 设置随机种子以确保结果可重现
np.random.seed(42)

# 生成扩展的分类数据集（200个样本）
# 特征：身高（160-190cm），体重（50-90kg），标签：0或1
n_samples = 200
height = np.random.uniform(160, 190, n_samples)
weight = np.random.uniform(50, 90, n_samples)
X = np.column_stack((height, weight))

# 随机将约3%的数据设置为NaN
mask = np.random.random(X.shape) < 0.03
X[mask] = np.nan

# 生成标签：基于简单的规则（例如，身高+体重>250则为1，否则为0）并添加噪声
y = np.where(np.nansum(X, axis=1) > 250, 1, 0)
# 添加一些随机噪声
noise = np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1])
y = np.logical_xor(y, noise).astype(int)

# 划分训练集和测试集（90%训练，10%测试）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 创建 Pipeline，整合 SimpleImputer、StandardScaler 和 LogisticRegression
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42))
])

# 定义 GridSearchCV 的超参数网格
param_grid = {
    'classifier__C': [0.1, 1.0, 10.0],  # 正则化参数
    'classifier__solver': ['lbfgs', 'liblinear']  # 优化器
}

# 初始化 GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  # 5 折交叉验证
    scoring='accuracy',
    n_jobs=-1  # 使用所有可用 CPU 核心
)

# 训练 GridSearchCV
grid_search.fit(X_train, y_train)

# 获取最佳模型
best_model = grid_search.best_estimator_

# 使用 cross_val_score 进行额外的交叉验证评估
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')

# 预测训练集和测试集
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# 计算准确率
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 计算测试集的 ROC 曲线和 AUC
y_test_score = best_model.predict_proba(X_test)[:, 1]  # 获取正类的概率
fpr, tpr, _ = roc_curve(y_test, y_test_score)
roc_auc = auc(fpr, tpr)

# 打印结果
print("\n交叉验证准确率（每折）：")
print(cv_scores)
print("\n平均交叉验证准确率：")
print(f"{np.mean(cv_scores):.2f} ± {np.std(cv_scores):.2f}")
print("\n训练集准确率：")
print(f"{train_accuracy:.2f}")
print("\n测试集准确率：")
print(f"{test_accuracy:.2f}")
print("\n测试集分类报告：")
print(classification_report(y_test, y_test_pred))
print("\n最佳超参数：")
print(grid_search.best_params_)
print("\nGridSearchCV 最佳交叉验证准确率：")
print(f"{grid_search.best_score_:.2f}")
print("\n测试集 AUC 值：")
print(f"{roc_auc:.2f}")

# 绘制 ROC 曲线
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## 高级实践案例

### 典型的分类任务工作流示例

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. 数据加载
digits = load_digits()
X, y = digits.data, digits.target

# 2. 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 数据预处理（标准化）
# 使用StandardScaler标准化特征（均值0，方差1），以提高LogisticRegression的性能
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 拟合并转换训练数据
X_test_scaled = scaler.transform(X_test)       # 转换测试数据

# 4. 模型训练与超参数调优
model = LogisticRegression(max_iter=1000, random_state=42)
# 通过GridSearchCV调优超参数C（正则化强度）和solver（优化算法）
param_grid = {'C': [0.1, 1, 10], 'solver': ['lbfgs', 'liblinear']}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)  # 训练模型

# 5. 预测与评估
y_pred = grid_search.predict(X_test_scaled)  # 预测
# 使用accuracy_score计算准确率，classification_report提供详细的性能指标（精确率、召回率、F1分数）
accuracy = accuracy_score(y_test, y_pred)    # 计算准确率
print("Best Parameters:", grid_search.best_params_)
print("Test Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

### 回归任务工作流示例

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 1. 数据加载
# load_diabetes 是一个回归数据集，包含 442 个样本，10 个特征（如年龄、BMI），目标是疾病进展的量化指标
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# 2. 数据集拆分
# 将数据集分为 80% 训练集和 20% 测试集，设置 random_state 确保结果可重现
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 数据预处理
# 使用 StandardScaler 标准化特征，使每个特征的均值为 0，方差为 1，有助于提高模型性能
scaler = StandardScaler()
# 对训练数据进行拟合（计算均值和标准差）并转换
X_train_scaled = scaler.fit_transform(X_train)
# 对测试数据仅进行转换（使用训练数据的均值和标准差）
X_test_scaled = scaler.transform(X_test)

# 4. 模型训练与超参数调优
# 初始化 RandomForestRegressor，设置 random_state 确保结果可重现
model = RandomForestRegressor(random_state=42)
# 定义超参数网格，用于 GridSearchCV 搜索最优参数
param_grid = {
    'n_estimators': [50, 100],  # 树的数量
    'max_depth': [None, 10],    # 树的最大深度
    'min_samples_split': [2, 5] # 节点分裂的最小样本数
}
# 使用 GridSearchCV 进行 5 折交叉验证，优化均方误差（负值形式）
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
# 训练模型，自动搜索最佳超参数
grid_search.fit(X_train_scaled, y_train)

# 5. 预测与评估
# 使用最佳模型对测试集进行预测
y_pred = grid_search.predict(X_test_scaled)
# 计算均方误差（MSE），评估预测误差
mse = mean_squared_error(y_test, y_pred)
# 计算 R² 分数，评估模型解释目标变量的程度
r2 = r2_score(y_test, y_pred)
# 输出最佳超参数、MSE 和 R² 分数
print("Best Parameters:", grid_search.best_params_)
print("Test Mean Squared Error:", mse)
print("Test R² Score:", r2)

### Pipeline示例：修改上面的回归任务，基于Pipeline完成

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# 1. 数据加载
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# 2. 数据集拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 构建Pipeline
# Pipeline整合预处理（标准化）和模型训练（RandomForestRegressor）
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 标准化特征，使均值为 0，方差为 1
    ('model', RandomForestRegressor(random_state=42))  # 随机森林回归模型
])

# 4. 超参数调优
# 定义超参数网格，注意使用Pipeline的参数命名格式（如 'model__参数名'）
param_grid = {
    'model__n_estimators': [50, 100],  # 树的数量
    'model__max_depth': [None, 10],    # 树的最大深度
    'model__min_samples_split': [2, 5] # 节点分裂的最小样本数
}
# 使用 GridSearchCV 进行 5 折交叉验证，优化均方误差（负值形式）
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# 5. 训练 Pipeline
# Pipeline自动按顺序执行scaler.fit_transform和model.fit
grid_search.fit(X_train, y_train)

# 6. 预测与评估
# 使用最佳模型对测试集进行预测，Pipeline自动应用scaler.transform和model.predict
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)  # 计算均方误差（MSE），评估预测误差
r2 = r2_score(y_test, y_pred)  # 计算 R² 分数，评估模型解释目标变量的程度
print("Best Parameters:", grid_search.best_params_)  # 输出最佳超参数、MSE 和 R² 分数
print("Test Mean Squared Error:", mse)
print("Test R² Score:", r2)

### 聚类和降维的简单示例
- 运行代码后，会生成一个散点图，展示降维后的数据点（2维），每个点根据 KMeans 的簇标签着色。
- 由于load_digits数据包含10个数字类别，散点图通常会显示10个簇的分布，但簇的分离程度取决于KMeans和PCA的效果。
- 输出示例（散点图）无法直接以文本形式展示，但点分布会反映数据的自然分组，颜色区分不同簇。

In [ ]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans  # KMeans 聚类算法
from sklearn.decomposition import PCA  # 主成分分析（PCA）降维算法
import matplotlib.pyplot as plt

# 1. 数据加载
X, y = load_digits(return_X_y=True)

# 2. 数据预处理
# 标准化有助于提高KMeans和PCA的性能，因为它们对特征尺度敏感
scaler = StandardScaler()
# fit_transform：对训练数据计算均值和标准差，并将数据转换为标准化形式
X_scaled = scaler.fit_transform(X)

# 3. 聚类：使用 KMeans 算法
# KMeans将数据分为指定数量的簇（n_clusters=10，对应 0-9 十个数字）
# random_state=42确保结果可重现，n_clusters=10是基于数据集的数字类别数
kmeans = KMeans(n_clusters=10, random_state=42)
# fit_predict：拟合KMeans模型并返回每个样本的簇标签
# X_scaled是标准化后的数据，KMeans根据欧氏距离最小化簇内方差
labels = kmeans.fit_predict(X_scaled)

# 4. 降维：使用 PCA 算法
# PCA将64维特征降到2维，以便在二维平面可视化
# n_components=2表示保留2个主成分，捕捉数据的主要方差
pca = PCA(n_components=2)
# fit_transform：计算主成分并将数据投影到前两个主成分上
X_reduced = pca.fit_transform(X_scaled)

# 5. 可视化
# 使用matplotlib绘制散点图，展示降维后的数据点
# X_reduced[:, 0]和X_reduced[:, 1]分别是第一和第二主成分
# c=labels表示用KMeans的簇标签为点着色，cmap='viridis'定义颜色映射
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, cmap='viridis')
plt.title("KMeans Clustering after PCA")
plt.show()

### 数据集拆分示例
- 第一次拆分生成训练+验证集（80%）和测试集（20%）。
- 第二次拆分将训练+验证集再分为训练集（60%）和验证集（20%）。

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# 加载数据
X, y = load_breast_cancer(return_X_y=True)

# 第一次拆分：80% 训练+验证，20% 测试
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 第二次拆分：从训练+验证中分出 25% 作为验证集（即总数据的 20%）
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

print("Train set size:", len(X_train))  # 60% of total
print("Validation set size:", len(X_val))  # 20% of total
print("Test set size:", len(X_test))  # 20% of total

### 交叉验证

#### cross_val_score函数

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score # 导入cross_val_score函数，用于交叉验证

# 加载鸢尾花数据集，X表示特征数据，y表示目标（标签）数据
X, y = load_iris(return_X_y=True) 
# 创建一个逻辑回归模型实例，并设置最大迭代次数为1000
model = LogisticRegression(max_iter=1000) 
# 使用交叉验证评估模型性能，cv=5表示5折交叉验证，scoring='accuracy'表示使用准确率作为评估指标
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy') 
print("Cross-Validation Accuracy:", scores.mean(), "±", scores.std()) # 打印交叉验证的平均准确率和标准差

#### cross_validate函数

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

X, y = load_iris(return_X_y=True) 
model = LogisticRegression(max_iter=1000) 

# 换用cross_validate函数，并指定accuracy和f1_macro两个指标
scores = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'f1_macro'], return_train_score=True)
print("Validation Accuracy:", scores['test_accuracy'].mean())
print("Train Accuracy:", scores['train_accuracy'].mean())

#### 交叉验证与Pipeline相结合

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# 加载鸢尾花数据集
X, y = load_iris(return_X_y=True)

# 构建机器学习工作流Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 第一步: 特征标准化
    ('svc', SVC())                 # 第二步: SVC，Pipeline中的最终模型，用于执行分类任务
])

# 定义交叉验证策略
# StratifiedKFold是一种分层K折交叉验证，特别适用于分类任务和类别不平衡的数据集
# 它确保每个折叠中各类别的样本比例与原始数据集保持一致
cv = StratifiedKFold(n_splits=5,     # 将数据集分成5个折叠 (K=5)
                     shuffle=True,   # 在分割前打乱数据，以增加随机性
                     random_state=42)# 设置随机种子，确保每次运行分割结果一致，方便复现

# 执行交叉验证
# cross_val_score函数用于在指定交叉验证策略下评估模型的性能
scores = cross_val_score(pipeline,  # 要评估的机器学习模型或Pipeline
                         X,         # 特征数据
                         y,         # 目标数据
                         cv=cv,     # 指定使用的交叉验证策略
                         scoring='accuracy', # 评估指标，这里使用准确率
                         n_jobs=-1) # 使用所有可用的CPU核心进行并行计算，加速运行

# 输出交叉验证结果
print("Cross-Validation Scores:", scores) # 打印每个折叠的准确率得分
print("Mean Accuracy:", scores.mean(),    # 打印所有折叠的平均准确率
      "±", scores.std())                # 打印准确率的标准差，表示结果的稳定性

### GridSearchCV示例

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 加载数据集
iris = datasets.load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print(f"特征名称: {feature_names}")
print(f"目标类别名称: {target_names}")
print(f"数据集形状: X={X.shape}, y={y.shape}")

# 2. 数据分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"\n训练集形状: {X_train.shape}, 测试集形状: {X_test.shape}")

# 3. 特征缩放 (SVM 对特征尺度敏感，必须进行缩放)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. 实例化和训练 SVC 模型 (使用默认 RBF 核)
print("\n--- 默认参数 SVC 模型 ---")
svc_default = SVC(random_state=42)
svc_default.fit(X_train_scaled, y_train)

# 5. 模型预测与评估
y_pred_default = svc_default.predict(X_test_scaled)
accuracy_default = accuracy_score(y_test, y_pred_default)
print(f"默认 SVC 模型准确率: {accuracy_default:.4f}")
print("\n默认 SVC 模型分类报告:\n", classification_report(y_test, y_pred_default, target_names=target_names))
print("默认 SVC 模型混淆矩阵:\n", confusion_matrix(y_test, y_pred_default))


# 6. 参数调优 (GridSearchCV)
print("\n--- 使用 GridSearchCV 进行参数调优 ---")
# 定义参数网格
param_grid = {
    'C': [0.1, 1, 10, 100],            # 惩罚参数
    'kernel': ['linear', 'rbf'],       # 核函数
    'gamma': ['scale', 'auto', 0.1, 1] # 核系数 (仅 RBF 核相关)
}

# 实例化 GridSearchCV
# cv=5 表示使用 5 折交叉验证
# n_jobs=-1 表示使用所有可用的 CPU 核心并行计算
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# 在缩放后的训练数据上执行网格搜索（包含内层的交叉验证过程）
grid_search.fit(X_train_scaled, y_train)

# 获取最佳超参数和模型
print(f"\n最佳参数组合: {grid_search.best_params_}")
print(f"最佳交叉验证准确率: {grid_search.best_score_:.4f}")
best_svc = grid_search.best_estimator_

# 使用最佳参数的模型进行预测和评估（在独立的测试集上进行最终评估）
y_pred_best = best_svc.predict(X_test_scaled)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"最佳 SVC 模型在测试集上的准确率: {accuracy_best:.4f}")
print("\n最佳 SVC 模型分类报告:\n", classification_report(y_test, y_pred_best, target_names=target_names))
print("最佳 SVC 模型混淆矩阵:\n", confusion_matrix(y_test, y_pred_best))


# 7. 可视化 (仅限二维数据，这里为了演示，只取两个特征)
# 为了简化可视化，我们只使用前两个特征
X_reduced = X[:, :2] # 只取 sepal length 和 sepal width
X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(
    X_reduced, y, test_size=0.3, random_state=42, stratify=y
)

# 对简化后的数据进行缩放
scaler_reduced = StandardScaler()
X_train_scaled_reduced = scaler_reduced.fit_transform(X_train_reduced)
X_test_scaled_reduced = scaler_reduced.transform(X_test_reduced)

# 使用最佳参数的 SVC 模型在二维数据上训练
best_svc_reduced = SVC(C=best_svc.C, kernel=best_svc.kernel, gamma=best_svc.gamma, random_state=42)
best_svc_reduced.fit(X_train_scaled_reduced, y_train_reduced)

# 绘制决策边界
def plot_decision_boundary(X, y, model, title):
    h = .02  # 网格中的步长
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(10, 7))
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k', s=60)
    
    # 绘制支持向量
    if hasattr(model, 'support_vectors_'):
        plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s=150,
                    facecolors='none', edgecolors='green', linewidth=2, label='Support Vectors')
    
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.title(title)
    plt.legend()
    plt.show()

plot_decision_boundary(X_train_scaled_reduced, y_train_reduced, best_svc_reduced, "SVC Decision Boundary (Iris - 2 Features)")

#### GridSearchVC、交叉验证和Pipeline的组合示例

In [ ]:
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 1. 加载鸢尾花数据集
X, y = load_iris(return_X_y=True)
# 分割训练集和测试集（80% 训练，20% 测试）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. 构建 Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 标准化：将特征缩放到均值为 0，方差为 1
    ('svc', SVC())                 # 支持向量机分类器
])

# 3. 定义超参数网格
param_grid = {
    'scaler__with_mean': [True, False],   # 标准化是否中心化
    'scaler__with_std': [True, False],    # 标准化是否缩放
    'svc__C': [0.1, 1, 10],              # SVM 正则化参数
    'svc__kernel': ['linear', 'rbf'],     # 核函数
    'svc__gamma': ['scale', 'auto', 0.1]  # 核函数系数
}

# 4. 定义交叉验证策略
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 5. 初始化 GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy',  # 使用准确率作为评估指标
    n_jobs=-1,           # 使用所有 CPU 核心加速
    verbose=1            # 输出搜索进度
)

# 6. 在训练集上执行超参数调优和交叉验证
grid_search.fit(X_train, y_train)

# 7. 输出结果
print("\n=== 超参数调优结果 ===")
print("最佳参数:", grid_search.best_params_)
print("最佳交叉验证准确率:", grid_search.best_score_)
print("最佳模型:", grid_search.best_estimator_)

# 8. 在测试集上进行预测和评估
y_pred = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("\n=== 测试集评估 ===")
print("测试集准确率:", test_accuracy)
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=load_iris().target_names))

# 9. 可视化交叉验证结果（可选）
# 输出所有参数组合的平均交叉验证评分
print("\n=== 所有参数组合的交叉验证评分 ===")
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"参数: {params}, 平均准确率: {mean_score:.4f}")

#### 上面示例的改进和对比版本
- 添加特征选择：在Pipeline中加入SelectKBest进行特征选择，优化选择特征数量
- 多指标评估：使用GridSearchCV的多指标评估（accuracy和f1_macro），并通过refit指定主要指标
- 使用RandomizedSearchCV：展示如何替换GridSearchCV，使用随机搜索优化参数空间
- 可视化结果：绘制交叉验证评分的分布图，分析参数组合的性能

代码说明
- RandomizedSearchCV：
  - 使用 uniform 分布抽样 C 和 gamma，探索连续参数空间。
  - n_iter=20 减少计算成本（20×5=100 次训练 vs. GridSearchCV 的 216×5=1080 次）。
- 多指标评估：
  - GridSearchCV同时评估accuracy和f1_macro，提供更全面的性能分析。
  - f1_macro适合不平衡数据集（尽管鸢尾花数据集较为平衡）。

In [ ]:
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import uniform
import numpy as np
import matplotlib.pyplot as plt

# 1. 加载鸢尾花数据集
X, y = load_iris(return_X_y=True)
# 分割训练集和测试集（80% 训练，20% 测试）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. 构建 Pipeline（添加特征选择）
pipeline = Pipeline([
    ('scaler', StandardScaler()),              # 标准化
    ('select', SelectKBest(score_func=f_classif)),  # 特征选择
    ('svc', SVC())                             # SVM 分类器
])

# 3. 定义超参数网格（用于 GridSearchCV）
param_grid = {
    'scaler__with_mean': [True, False],        # 标准化是否中心化
    'scaler__with_std': [True, False],         # 标准化是否缩放
    'select__k': [2, 3, 4],                   # 选择特征数量
    'svc__C': [0.1, 1, 10],                   # 正则化参数
    'svc__kernel': ['linear', 'rbf'],          # 核函数
    'svc__gamma': ['scale', 'auto', 0.1]      # 核函数系数
}

# 4. 定义参数分布（用于 RandomizedSearchCV）
param_dist = {
    'scaler__with_mean': [True, False],
    'scaler__with_std': [True, False],
    'select__k': [2, 3, 4],
    'svc__C': uniform(0.1, 10),                # 连续分布：0.1 到 10
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': uniform(0.01, 0.1)           # 连续分布：0.01 到 0.1
}

# 5. 定义交叉验证策略
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 6. 执行 GridSearchCV（多指标评估）
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring=['accuracy', 'f1_macro'],  # 多指标：准确率和 F1 分数（宏平均）
    refit='accuracy',                  # 以准确率选择最佳模型
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

# 7. 执行 RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=20,                         # 随机抽样 20 次
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)
random_search.fit(X_train, y_train)

# 8. 输出 GridSearchCV 结果
print("\n=== GridSearchCV 结果 ===")
print("最佳参数:", grid_search.best_params_)
print("最佳交叉验证准确率:", grid_search.best_score_)
print("最佳交叉验证 F1 分数:", grid_search.cv_results_['mean_test_f1_macro'][grid_search.best_index_])
print("测试集准确率:", accuracy_score(y_test, grid_search.predict(X_test)))
print("\nGridSearchCV 分类报告:")
print(classification_report(y_test, grid_search.predict(X_test), target_names=load_iris().target_names))

# 9. 输出 RandomizedSearchCV 结果
print("\n=== RandomizedSearchCV 结果 ===")
print("最佳参数:", random_search.best_params_)
print("最佳交叉验证准确率:", random_search.best_score_)
print("测试集准确率:", accuracy_score(y_test, random_search.predict(X_test)))
print("\nRandomizedSearchCV 分类报告:")
print(classification_report(y_test, random_search.predict(X_test), target_names=load_iris().target_names))

# 10. 可视化交叉验证评分分布（GridSearchCV）
plt.figure(figsize=(10, 6))
plt.hist(grid_search.cv_results_['mean_test_accuracy'], bins=20, alpha=0.5, label='GridSearchCV Accuracy', color='blue')
plt.hist(random_search.cv_results_['mean_test_score'], bins=20, alpha=0.5, label='RandomizedSearchCV Accuracy', color='orange')
plt.xlabel('Cross-Validation Accuracy')
plt.ylabel('Frequency')
plt.title('Distribution of Cross-Validation Accuracy Scores')
plt.legend()
plt.grid(True)
plt.show()

#### 基于RandomSearchVC、交叉验证和Pipeline的回归任务第二个示例

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import RandomizedSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from scipy.stats import randint, uniform

# 加载数据
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建 Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42))
])

# 定义参数分布
param_dist = {
    'scaler__with_std': [True, False],         # 标准化是否缩放
    'rf__n_estimators': randint(50, 200),      # 树数量
    'rf__max_depth': [None, 10, 20],          # 最大深度
    'rf__min_samples_split': randint(2, 10)    # 最小分裂样本数
}

# 定义交叉验证策略
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 初始化 RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_dist,
    n_iter=10,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)
random_search.fit(X_train, y_train)

# 输出结果
print("Best Parameters:", random_search.best_params_)
print("Best CV Score (neg MSE):", random_search.best_score_)
print("Test MSE:", mean_squared_error(y_test, random_search.predict(X_test)))

### 案例：基于Iris数据集的分类任务
- Iris数据集是一个经典的多分类数据集，包含150个样本，描述了三种鸢尾花（Setosa（山鸢尾）、Versicolor（蔓生鸢尾）、Virginica（弗吉尼亚鸢尾））的4个特征（萼片长度、萼片宽度、花瓣长度、花瓣宽度）。
- 任务：基于4个特征预测鸢尾花的类别（三分类问题）。
- 方法：使用Pipeline结合数据标准化（StandardScaler）和逻辑回归模型（LogisticRegression）进行分类。

In [ ]:
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 1. 加载数据集
# 使用 load_iris() 加载Iris数据集，得到特征矩阵X（150 行，4 列）和标签向量y（150 个标签，0/1/2 对应三种鸢尾花）
# 数据包含4个数值型特征，无需额外处理缺失值或类别编码
iris = load_iris()
X = iris.data  # 特征：(150, 4)
y = iris.target  # 标签：(150,)

# 2. 划分训练集和测试集
# 使用 train_test_split 将数据分为训练集（70%，105 个样本）和测试集（30%，45 个样本）
# 参数 test_size=0.3 表示测试集占比 30%，random_state=42 确保划分可重现
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. 创建 Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 标准化特征
    ('clf', LogisticRegression(random_state=42))  # 逻辑回归模型
])

# 4. 训练模型
pipeline.fit(X_train, y_train)

# 5. 预测
y_pred = pipeline.predict(X_test)

# 6. 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

#### 交叉验证
为了更稳健地评估模型性能，可以使用交叉验证代替单一的 train-test split。
- cross_val_score 将数据分为5折，每次使用4折训练、1折测试。
- 输出每折的准确率以及平均准确率和标准差，评估模型的稳定性。

In [ ]:
from sklearn.model_selection import cross_val_score

# 使用 5 折交叉验证评估 Pipeline
scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean CV accuracy: {scores.mean():.2f} ± {scores.std():.2f}")

### 案例：基于 MNIST 数据集的手写数字分类
- MNIST 是一个经典的手写数字数据集，包含70000张28×28像素的灰度图像，每张图像表示一个手写数字（0-9）。每个样本有 784 个特征（28×28 像素展平为向量），标签为对应的数字类别。
- 本示例的任务：基于图像的像素特征预测手写数字的类别（10 分类问题）。
下面是一个完整的代码示例，涵盖数据加载、预处理、模型训练、测试和评估。但是，为了简化计算，本示例中使用load_digits()（Scikit-Learn提供的8×8像素简化版MNIST数据集，1797个样本，64个特征）。如果需要使用完整的MNIST数据集（70000个样本，784个特征），可以替换为from sklearn.datasets import fetch_openml; X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)，但流程类似。
#### 下面代码的训练过程说明
训练过程由 Pipeline 自动管理，具体步骤如下：
1. 数据加载
   - 使用load_digits()加载简化版MNIST数据集，得到特征矩阵X（1797行，64列，每个样本是8×8像素展平后的向量）和标签向量 y（1797个标签，0-9）。
   - 每个特征表示像素的灰度值（0-16），无需处理缺失值或类别编码。
2. 数据集划分
   - 使用train_test_split将数据分为训练集（70%，1257 个样本）和测试集（30%，540 个样本）。
   - 参数test_size=0.3表示测试集占比 30%，random_state=42 确保划分可重现。
3. Pipeline配置
   - Pipeline 包含两个步骤
     - StandardScaler：标准化特征（将像素值缩放到均值为 0、方差为 1），使 SVM 模型更有效，因为 SVM 对特征尺度敏感。
     - SVC：支持向量分类器，使用径向基函数（RBF）核（kernel='rbf'），适合非线性可分数据。
   - Pipeline 确保训练和测试数据使用相同的标准化参数。
4. 训练
   - 步骤 1：StandardScaler.fit_transform(X_train)
     - 计算训练数据 X_train 每个特征的均值和标准差。
     - 对 X_train 应用标准化：(X - mean) / std，生成标准化的特征矩阵 X_train_scaled。
   - 步骤 2：SVC.fit(X_train_scaled, y_train)
     - 使用标准化后的训练数据 X_train_scaled 和标签 y_train 训练 SVM 模型。
     - SVM 学习支持向量和决策边界，优化分类超平面（RBF 核将数据映射到高维空间）。
#### 测试方法说明
测试过程用于评估模型在新数据上的性能，具体步骤如下：
1. 预测
   - 步骤 1：StandardScaler.transform(X_test)
     - 使用训练阶段学习的均值和标准差，对测试数据 X_test 进行标准化，生成 X_test_scaled。
     - 注意：测试数据只调用 transform()，不调用 fit()，以避免数据泄漏。
   - 步骤 2：SVC.predict(X_test_scaled)
     - 使用训练好的 SVM 模型对 X_test_scaled 进行预测，生成预测标签 y_pred。
     - SVM 输出每个样本的预测类别（0-9）。
2. 评估模型
   - 准确率（Accuracy）
     - 使用 accuracy_score(y_test, y_pred) 计算预测正确的比例。
     - 公式：accuracy = (正确预测的样本数) / (总样本数)。
   - 分类报告（Classification Report）
     - 使用 classification_report 输出每个类别的精确率（Precision）、召回率（Recall）和F1 分数。
     - 精确率：TP / (TP + FP)，表示预测为某类的样本中实际为该类的比例。
     - 召回率：TP / (TP + FN)，表示实际为某类的样本中被正确预测的比例。
     - F1 分数：2 * (Precision * Recall) / (Precision + Recall)。
   - 混淆矩阵（Confusion Matrix）
     - 使用 confusion_matrix 输出 10×10 矩阵，显示每个类别的预测情况（行是真实类别，列是预测类别）。
     - 对角线表示正确预测的样本数，非对角线表示错误预测。

In [ ]:
# 导入必要的库
from sklearn.datasets import load_digits  # 使用 digits 数据集（小型 MNIST）
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# 1. 加载数据集（使用 digits 数据集，简化版 MNIST）
digits = load_digits()
X = digits.data  # 特征：(1797, 64)，8x8 像素展平
y = digits.target  # 标签：(1797,)，0-9 的类别

# 2. 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. 创建 Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 标准化特征
    ('clf', SVC(kernel='rbf', random_state=42))  # 支持向量机分类器
])

# 4. 训练模型
pipeline.fit(X_train, y_train)

# 5. 预测
y_pred = pipeline.predict(X_test)

# 6. 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

#### 上面代码的输出结果的说明
- Accuracy表示模型在测试集上的准确率，例如98%表明分类效果很好。
- 分类报告显示每个类别的精确率、召回率和 F1 分数都在 0.92-1.00 之间，模型性能均衡。
- 混淆矩阵显示大多数样本被正确分类，少数错误主要出现在的类别上，可能由于有些数字的形状相似所致（如8和3、9和5）。

### 扩展：交叉验证和参数调优
为了更稳健地评估模型性能并优化超参数，可以使用交叉验证和网格搜索。

In [ ]:
from sklearn.model_selection import GridSearchCV

# 定义参数网格
param_grid = {
    'clf__C': [0.1, 1, 10],  # SVM 的正则化参数
    'clf__gamma': ['scale', 0.01, 0.1]  # RBF 核的宽度参数
}

# 创建 GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# 训练和搜索最佳参数
grid_search.fit(X_train, y_train)

# 输出最佳参数和分数
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.2f}")

# 使用最佳模型预测
y_pred = grid_search.predict(X_test)
print(f"Test accuracy: {accuracy_score(y_test, y_pred):.2f}")

### 可视化
为了直观理解模型表现，下面的代码用于绘制混淆矩阵的热图

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 绘制混淆矩阵
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=digits.target_names, yticklabels=digits.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

### 回归模型示例

In [ ]:
# 导入必要的库
from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 使用 make_regression 生成一个包含 1000 个样本、10 个特征的合成回归数据集，带有少量噪声（noise=0.1）
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 数据标准化（SVR对特征尺度敏感）
# 使用 StandardScaler 对特征 $X$ 进行标准化（均值为 0，标准差为 1），因为 SVR 对特征尺度敏感
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# 如果目标变量y的尺度变化较大，也可对其标准化（可选）
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

# 创建并训练SVR模型（使用RBF核）
svr = SVR(kernel='rbf', C=1.0, epsilon=0.1, gamma='scale')
svr.fit(X_train, y_train)

# 进行预测
y_pred = svr.predict(X_test)

# 反标准化预测结果（如果对y标准化了）
y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).ravel()
y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()

# 评估模型
# 使用均方误差（MSE）评估预测误差。
# 使用 R^2 分数评估模型的解释能力（越接近 1 越好）。
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.2f}")

# 可视化预测结果（以第一个特征为例）
import matplotlib.pyplot as plt

plt.scatter(X_test[:, 0], y_test, color='blue', label='True values', alpha=0.5)
plt.scatter(X_test[:, 0], y_pred, color='red', label='Predicted values', alpha=0.5)
plt.xlabel('Feature 1 (Standardized)')
plt.ylabel('Target (Original Scale)')
plt.title('SVR: True vs Predicted Values')
plt.legend()
plt.show()

### 模型选择示例
模型选择（Model Selection）是指在机器学习任务中，从多个候选模型或同一模型的不同超参数配置中，选择性能最佳的模型或配置，以在给定任务上获得最优的预测能力。模型选择的目标是找到一个模型（或模型配置），在测试数据或未见过的数据上具有良好的泛化性能。模型选择通常涉及以下几个方面：
1. 选择模型类型：比较不同类型的机器学习算法，例如逻辑回归、支持向量机（SVM）、随机森林、神经网络等。
2. 选择超参数：为选定的模型调整超参数，例如 SVM 的正则化参数 C 和核函数参数 gamma，或随机森林的树数量 n_estimators。
3. 评估泛化性能：使用验证集或交叉验证（Cross-Validation）评估模型在未见过数据上的表现，避免过拟合。
4. 权衡计算成本与性能：在性能和计算复杂度之间找到平衡，特别是在大规模数据集（如 MNIST）上。
模型选择通常通过以下方法实现：
- 交叉验证：将训练数据分为多折（如 5 折或 10 折），在每折上训练和验证模型，计算平均性能指标（如准确率、F1 分数）。
- 网格搜索（Grid Search）：系统地测试超参数的组合，找到最优配置。
- 随机搜索（Random Search）：随机采样超参数组合，效率高于网格搜索，尤其在超参数空间较大时。
- 自动化模型选择：使用工具（如 AutoML 或 Scikit-Learn 的 RandomizedSearchCV）自动选择模型和超参数。

#### 在 MNIST 数据集分类任务上进行模型选择
MNIST 是一个图像分类任务，适合的模型包括：
- 逻辑回归（Logistic Regression）：简单、快速，适合线性可分数据。
- 支持向量机（SVM）：通过核函数（如 RBF）处理非线性关系，效果好但计算成本高。
- 随机森林（Random Forest）：集成方法，适合高维数据，训练和预测较快。
- K 近邻（K-Nearest Neighbors, KNN）：基于距离的非参数方法，适合小型数据集。
- （可选）神经网络：如多层感知机（MLP）或卷积神经网络（CNN），但 CNN 通常需要深度学习框架（如 TensorFlow/Keras）。

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 加载简化版 MNIST 数据集
digits = load_digits()
X = digits.data  # 特征：(1797, 64)
y = digits.target  # 标签：(1797,)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 定义候选模型及其超参数
pipelines = {
    'logistic': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    'svm': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', SVC(random_state=42))
    ]),
    'random_forest': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier(random_state=42))
    ])
}

# 定义超参数网格
param_grids = {
    'logistic': {
        'clf__C': [0.1, 1.0, 10.0],  # 正则化强度的倒数
        'clf__solver': ['lbfgs', 'liblinear']  # 优化算法
    },
    'svm': {
        'clf__C': [0.1, 1.0, 10.0],  # 正则化参数
        'clf__kernel': ['linear', 'rbf'],  # 核函数
        'clf__gamma': ['scale', 0.01]  # RBF 核的宽度
    },
    'random_forest': {
        'clf__n_estimators': [50, 100, 200],  # 树的数量
        'clf__max_depth': [None, 10, 20]  # 最大深度
    }
}

# 使用交叉验证选择最佳模型：对每个模型执行网格搜索，使用 5 折交叉验证评估性能
# 存储最佳模型和分数
best_models = {}
best_scores = {}

for model_name in pipelines:
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(
        pipelines[model_name],
        param_grids[model_name],
        cv=5,  # 5 折交叉验证
        scoring='accuracy',  # 使用准确率作为评估指标
        n_jobs=-1  # 使用所有 CPU 核心
    )
    grid_search.fit(X_train, y_train)
    
    # 保存最佳模型和分数
    best_models[model_name] = grid_search.best_estimator_
    best_scores[model_name] = grid_search.best_score_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best CV accuracy for {model_name}: {grid_search.best_score_:.2f}")

# 在测试集上评估并选出最佳模型
best_model_name = max(best_scores, key=best_scores.get)
best_model = best_models[best_model_name]
print(f"\nBest model: {best_model_name} with CV accuracy: {best_scores[best_model_name]:.2f}")

# 在测试集上预测
y_pred = best_model.predict(X_test)

# 评估测试集性能
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))    

### 机器学习在SRE中的应用示例
以下示例结合Pipeline、交叉验证和GridSearchCV，实现SRE领域的异常检测任务。假设我们有一个系统指标数据集（模拟CPU和内存使用率），目标是检测异常点。

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

# 1. 模拟系统指标数据（CPU 和内存使用率）
np.random.seed(42)
X_normal = np.random.normal(loc=0.5, scale=0.1, size=(100, 2))  # 正常数据
X_anomaly = np.random.uniform(low=0.9, high=1.0, size=(10, 2))   # 异常数据
X = np.vstack([X_normal, X_anomaly])
y = np.array([1] * 100 + [-1] * 10)  # 标签：1=正常，-1=异常（仅用于评估）

# 2. 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 构建 Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),           # 标准化
    ('isolation_forest', IsolationForest())  # 孤立森林异常检测
])

# 4. 定义超参数网格
param_grid = {
    'scaler__with_mean': [True, False],
    'isolation_forest__n_estimators': [50, 100, 200],  # 树数量
    'isolation_forest__contamination': [0.05, 0.1, 0.2]  # 异常比例
}

# 5. 初始化 GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,                   # 5 折交叉验证
    scoring='accuracy',     # 假设有标签评估
    n_jobs=-1,
    verbose=1
)

# 6. 训练模型
grid_search.fit(X_train, y_train)

# 7. 输出结果
print("\n=== GridSearchCV 结果 ===")
print("最佳参数:", grid_search.best_params_)
print("最佳交叉验证准确率:", grid_search.best_score_)
print("测试集准确率:", grid_search.score(X_test, y_test))

# 8. 可视化结果
import matplotlib.pyplot as plt
y_pred = grid_search.predict(X_test)
plt.scatter(X_test[:, 0], X_test[:, 1], c=['red' if x == -1 else 'blue' for x in y_pred])
plt.xlabel('CPU Usage')
plt.ylabel('Memory Usage')
plt.title('Anomaly Detection Results')
plt.show()